# Introducción a Pytest para Data Engineers

## ¿Qué es Pytest?

Pytest es un framework de testing para Python que se ha convertido en el estándar de facto para realizar pruebas en proyectos de Python. A diferencia de otros frameworks como unittest (que viene incluido en la biblioteca estándar de Python), pytest ofrece una sintaxis más sencilla y potente, permitiendo escribir tests más concisos y expresivos.

Para los ingenieros de datos, pytest representa una herramienta fundamental en el desarrollo de pipelines de datos robustos y confiables. En un entorno donde la calidad de los datos y la precisión de las transformaciones son críticas, contar con pruebas automatizadas se vuelve esencial para garantizar que nuestros procesos funcionen correctamente y produzcan los resultados esperados.

## ¿Por qué es importante el testing en Data Engineering?

En el campo de la ingeniería de datos, trabajamos constantemente con flujos de datos complejos que pueden incluir múltiples fuentes, transformaciones y destinos. Cualquier error en estos procesos puede propagarse y amplificarse, llevando a decisiones incorrectas basadas en datos erróneos. Algunos motivos por los que el testing es crucial en Data Engineering incluyen:

1. **Validación de la calidad de datos**: Nos permite verificar que los datos cumplen con los criterios de calidad establecidos (completitud, precisión, consistencia, etc.).

2. **Verificación de transformaciones**: Asegura que nuestras funciones de transformación de datos producen los resultados esperados bajo diferentes escenarios.

3. **Detección temprana de errores**: Identificar problemas en etapas tempranas del desarrollo, cuando son más fáciles y menos costosos de corregir.

4. **Facilita la refactorización**: Permite modificar y mejorar el código con confianza, sabiendo que cualquier regresión será detectada por las pruebas.

5. **Documentación viva**: Los tests actúan como documentación ejecutable que muestra cómo se espera que funcione el código.

## Ventajas de Pytest sobre otros frameworks

Pytest ha ganado popularidad por varias razones que lo hacen especialmente adecuado para proyectos de Data Engineering:

- **Sintaxis simple**: No requiere clases de test complejas, permitiendo escribir tests como simples funciones.
- **Fixtures potentes**: Mecanismo flexible para preparar y limpiar el entorno de pruebas, ideal para trabajar con conjuntos de datos.
- **Parametrización**: Facilita ejecutar la misma prueba con diferentes conjuntos de datos de entrada.
- **Plugins extensos**: Ecosistema rico de plugins para integración con otras herramientas y frameworks (pandas, SQL, Spark, etc.).
- **Informes detallados**: Proporciona información clara sobre fallos de tests, facilitando la depuración.
- **Marcadores**: Permite categorizar y seleccionar tests específicos para ejecución.

## Lo que aprenderemos en este tutorial

A lo largo de estos notebooks, exploraremos cómo utilizar pytest para verificar la calidad y el procesamiento de datos en un contexto de Data Engineering. Cubriremos:

1. **Configuración básica**: Instalación y estructura de proyectos con pytest.
2. **Escritura de tests**: Sintaxis y convenciones para crear tests efectivos.
3. **Fixtures**: Cómo preparar datos de prueba reutilizables.
4. **Parametrización**: Testing con múltiples conjuntos de datos.
5. **Testing de funciones de procesamiento de datos**: Verificación de transformaciones y cálculos.
6. **Validación de datos**: Tests para verificar la calidad e integridad de los datos.
7. **Integración con pandas y otras bibliotecas de datos**: Estrategias específicas para testing con estas herramientas.

Al finalizar este tutorial, tendrás las habilidades necesarias para implementar una estrategia de testing efectiva en tus proyectos de Data Engineering, mejorando la calidad y confiabilidad de tus pipelines de datos.

## Requisitos previos

Para seguir este tutorial, necesitarás:

- Python 3.6 o superior
- Conocimientos básicos de Python
- Familiaridad con conceptos de Data Engineering
- Entorno de desarrollo (Jupyter, VSCode, PyCharm, etc.)

Vamos a instalar las bibliotecas necesarias:

In [ ]:
# Instalación de las bibliotecas necesarias
# !pip install pytest pytest-cov pandas numpy

## Estructura del proyecto

Para este tutorial, trabajaremos con una estructura de proyecto típica para Data Engineering:

```
pytest_tutorial/
├── data/                  # Datos de ejemplo
│   └── ventas_productos.csv
├── notebooks/             # Jupyter notebooks para el tutorial
├── utils/                 # Módulos de utilidades y funciones
│   ├── __init__.py
│   ├── data_processing.py # Funciones para procesar datos
│   └── data_validation.py # Funciones para validar datos
└── tests/                 # Tests de pytest
    ├── __init__.py
    ├── test_processing.py # Tests para funciones de procesamiento
    └── test_validation.py # Tests para funciones de validación
```

Esta estructura nos permitirá separar claramente el código de producción (en `utils/`) de los tests (en `tests/`), siguiendo las mejores prácticas de desarrollo de software.

## Explorando nuestro dataset

Antes de sumergirnos en pytest, echemos un vistazo al dataset con el que trabajaremos en este tutorial. Se trata de un conjunto de datos de ventas de productos electrónicos que utilizaremos para ilustrar cómo testear funciones de procesamiento y validación de datos.

In [1]:
import pandas as pd

# Cargamos el dataset
df = pd.read_csv('../data/ventas_productos.csv')

# Mostramos las primeras filas
df.head()

,id,fecha,producto,categoria,precio,cantidad,descuento,total
0,1,2023-01-05,Laptop HP,Electrónica,899.99,1,0.05,854.99
1,2,2023-01-10,Monitor Dell,Electrónica,249.99,2,0.00,499.98
2,3,2023-01-15,Teclado Logitech,Accesorios,59.99,3,0.10,161.97
3,4,2023-01-20,Mouse Inalámbrico,Accesorios,29.99,5,0.00,149.95
4,5,2023-01-25,Disco SSD 500GB,Almacenamiento,89.99,2,0.15,152.98


In [2]:
# Información básica del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         20 non-null     int64  
 1   fecha      20 non-null     object 
 2   producto   20 non-null     object 
 3   categoria  20 non-null     object 
 4   precio     20 non-null     float64
 5   cantidad   20 non-null     int64  
 6   descuento  20 non-null     float64
 7   total      20 non-null     float64
dtypes: float64(3), int64(2), object(3)
memory usage: 1.4+ KB


In [3]:
# Estadísticas descriptivas
df.describe()

,id,precio,cantidad,descuento,total
count,20.00000,20.000000,20.000000,20.000000,20.000000
mean,10.50000,178.140000,2.250000,0.075000,230.452500
std,5.91608,204.980172,1.371707,0.067862,183.257463
min,1.00000,12.990000,1.000000,0.000000,64.950000
25%,5.75000,57.490000,1.000000,0.000000,123.987500
50%,10.50000,89.990000,2.000000,0.050000,157.475000
75%,15.25000,249.990000,3.000000,0.112500,289.732500
max,20.00000,899.990000,5.000000,0.200000,854.990000


## Conclusión

En esta introducción, hemos presentado pytest como una herramienta fundamental para el testing en proyectos de Data Engineering. Hemos discutido por qué el testing es crucial en este campo y las ventajas que ofrece pytest sobre otros frameworks.

También hemos explorado brevemente el dataset que utilizaremos a lo largo del tutorial y la estructura del proyecto que seguiremos.

En los siguientes notebooks, profundizaremos en los conceptos y técnicas específicas de pytest, aprendiendo a escribir tests efectivos para nuestras funciones de procesamiento y validación de datos.

¡Continuemos con el siguiente notebook para aprender los fundamentos de pytest y cómo escribir nuestros primeros tests!